In [10]:
["time", "avg","min","max","gap","median"][1::]

['avg', 'min', 'max', 'gap', 'median']

In [11]:
%matplotlib inline
%config InlineBackend.figure_format ='retina'
import pandas as pd
import numpy as np
import sys
from io import StringIO
import glob
if not hasattr(sys, "version_info") or sys.version_info < (3, 5):
  raise SystemExit("This program requires Python 3.5 or later.")
import csv
import matplotlib
#matplotlib.use('nbagg')
import matplotlib.pyplot as plt
import os
## https://github.com/ipython/ipyparallel
import ipyparallel as ipp
from IPython.display import JSON
import json
import statistics as st
from pympler import asizeof
from hurry.filesize import size
data_sources = glob.glob("data/*_shops.xml")

import tabletext
# tags

workers = ipp.Client()[:]


In [12]:
def unic(l: list) -> list :
    return list(set(l))

In [13]:
def processSnapshot(fname):
    import xml.etree.ElementTree as ET
    import statistics as st
    
    def processShop(tree, time):
        shopData = []
        ttidList = []
        for good in tree:
            goodData = {}
            goodData["ttid"] = int(good.findall("ttid")[0].text)
            ttidList.append(goodData["ttid"])
            goodData["time"] = time
            goodData["shop"] = tree.attrib["name"]
            goodData["shopid"] = tree.attrib["id"]
            goodData["shopmoney"] = tree.attrib["money"]
            sellElems = good.findall("sell")
            if sellElems:
                goodData["sell"] = float(sellElems[0].text)
            else:
                goodData["sell"] = 0.0

            buyElems = good.findall("buy")
            if buyElems:
                goodData["buy"] = float(buyElems[0].text)
            else:
                goodData["buy"] = 0.0
            goodData["count"] = int(good.findall("count")[0].text)
            goodData["thingname"] = good.findall("thingname")[0].text
            shopData.append(goodData)
        return (shopData, ttidList)
    

    
    timetag = "lastupdatetime"
    shopTag = "shop"
    xmltree = ET.parse(fname)
    xmlroot = xmltree.getroot()
    timeelem = xmlroot.find(timetag)
    dataTime = int(timeelem.text)
    orders = []
    ttids = []
    for shop in xmlroot.findall(shopTag):
        rc = processShop(shop, dataTime)
        shopData = rc[0]
        print(shopData)
        ttidList = rc[1]
        orders.extend(shopData)
        ttids.extend(ttidList)
    ttid_summary = []
    for ttid in ttids:
        ttid_stat = {}
        ttid_stat["ttid"] = ttid
        ttid_stat["time"] = dataTime
        ttid_stat["buy"] = []
        ttid_stat["sell"] = []
        ttid_stat["capacity"] = 0
        for order in orders:
            if order["ttid"] == ttid:
                ttid_stat["capacity"] += order["count"]
                if order["buy"] > 0:
                    ttid_stat["buy"].append(order["buy"] )
                if order["sell"] > 0:
                    ttid_stat["sell"].append(order["sell"] )  
        
        ttid_stat["buystat"] = {}
        if ttid_stat["buy"]:
            ttid_stat["buystat"]["avg"] = st.mean(ttid_stat["buy"])
            ttid_stat["buystat"]["min"] = min(ttid_stat["buy"])
            ttid_stat["buystat"]["max"] = max(ttid_stat["buy"])
            ttid_stat["buystat"]["gap"] = ttid_stat["buystat"]["max"] - ttid_stat["buystat"]["min"]
            ttid_stat["buystat"]["median"] = st.median(ttid_stat["buy"])
            ttid_stat["buystat"]["stdev"] = 0
            if len(ttid_stat["buy"]) >= 2:
                ttid_stat["buystat"]["stdev"] = st.stdev(ttid_stat["buy"], ttid_stat["buystat"]["avg"])

        ttid_stat["sellstat"] = {}
        if len(ttid_stat["sell"]) >= 2:
            ttid_stat["sellstat"]["avg"] = st.mean(ttid_stat["sell"])
            ttid_stat["sellstat"]["min"] = min(ttid_stat["sell"])
            ttid_stat["sellstat"]["max"] = max(ttid_stat["sell"])
            ttid_stat["sellstat"]["gap"] = ttid_stat["sellstat"]["max"] - ttid_stat["sellstat"]["min"]
            ttid_stat["sellstat"]["median"] = st.median(ttid_stat["sell"])
            ttid_stat["sellstat"]["stdev"] = 0
            if len(ttid_stat["sell"]) >= 2:
                ttid_stat["sellstat"]["stdev"] = st.stdev(ttid_stat["sell"], ttid_stat["sellstat"]["avg"])
        
        ttid_summary.append(ttid_stat)
    return (ttid_summary, ttids)

In [14]:
%%time
full_pathes = [("%s/%s" % (os.getcwd(), f)) for f in data_sources]
parts = workers.map_async(processSnapshot, full_pathes).get()
full_pathes = None
print("data size %s" % size(asizeof.asizeof(parts)))

data size 3G
CPU times: user 3min 27s, sys: 28.3 s, total: 3min 56s
Wall time: 8min 43s


In [15]:
%%time
ttids = []
stats = []
for part in parts:
    stats.extend(part[0])
    ttids.extend(part[1])
parts = None
ttids = unic(ttids)
print("data size %s" % size(asizeof.asizeof(stat)))

data size 3G
CPU times: user 2min 44s, sys: 2.58 s, total: 2min 46s
Wall time: 2min 46s


In [21]:
frame_source = []
for stat in stats:
    if stat["sellstat"]:
        record = [
            stat["time"],
            stat["sellstat"]["avg"],
            stat["sellstat"]["min"],
            stat["sellstat"]["max"],
            stat["sellstat"]["gap"],
            stat["sellstat"]["median"]
        ]
        frame_source.append(record)
print(len(frame_source))
df = pd.DataFrame(frame_source)
headers = ["time", "avg","min","max","gap","median"]
df.columns = headers

df.plot.area(x=headers[0], y=headers[1::])
plt.title("total")
plt.grid(b=True, which='minor', color='#2c3e50')
plt.minorticks_on()
plt.show()

1151426


AttributeError: 'list' object has no attribute 'is_unique'